# Create Pauli Measurement Function

In [93]:
%%writefile Define_Pauli_Measure.py


from qiskit import Aer
from qiskit.visualization import *
from qiskit import quantum_info as qi
import copy
from Define_Translator import translate, measure_rho
import numpy as np
from Define_Paulis import Mdot

def measure_pauli(p_label,psi0,method = 'simple'):
    
    #Simple method of calculation####
    if method == 'simple':
        rho = translate(psi0)
        Op = qi.Operator.from_label(p_label)
        return np.trace( Mdot([Op,rho]) )
    #################################
        
    #apply rotations#################
    psi = copy.deepcopy(psi0)
    pauli_qs = []
    Z_label = ''
    Q = len(p_label)
    for q,p in enumerate(p_label):
        if p == 'X':
            psi.ry(-np.pi/2,Q-1-q)
            pauli_qs.append(1)
            Z_label += 'Z'
        if p == 'Y':
            psi.rx(np.pi/2,Q-1-q)
            pauli_qs.append(1)
            Z_label += 'Z'
        if p == 'Z':
            pauli_qs.append(1)
            Z_label += 'Z'
        if p == 'I':
            pauli_qs.append(0)
            Z_label += 'I'
    ################################
    
    #Using matrix multiplication####
    if method == 'matrix':
        # add results
        rho = translate(psi)
        r = measure_rho(rho)
        z_measure = 0
        for key in list(r.keys()):
            n = 0
            for q in range(len(key)):
                if key[q] == '1' and pauli_qs[q] == 1:
                    n += 1
            z_measure += (-1)**n * r[key] 
        return z_measure
    #################################
    
    #Using the qasm simulator########
    if method == "simulator":
        sim = Aer.get_backend("qasm_simulator")
        psi.measure(psi.qubits,psi.clbits)
        r = execute(psi, backend = sim,shots = 10000).result().get_counts()
        z_measure = 0
        total = 0
        for key in list(r.keys()):
            n = 0
            for q in range(len(key)):
                if key[q] == '1' and pauli_qs[q] == 1:
                    n += 1
            z_measure += (-1)**n * r[key] 
            total += r[key]
        return z_measure/total
    ###################################
    
    raise NameError(method + ' is not a recognized method')
    return method + ' is not a recognized method'

def measure_E(H_paulis,cir,method = 'simple'):
    paulis = list(H_paulis.keys())
    e = 0
    for p in paulis:
        #print(p, H_paulis[p] , measure_pauli(p,cir,method))
        ep = H_paulis[p] * measure_pauli(p,cir,method)
        e += ep
    return e

def apply_rotations(p_label,psi0):
    psi = copy.deepcopy(psi0)
    pauli_qs = []
    Z_label = ''
    Q = len(p_label)
    for q,p in enumerate(p_label):
        if p == 'X':
            psi.ry(-np.pi/2,Q-1-q)
            pauli_qs.append(1)
            Z_label += 'Z'
        if p == 'Y':
            psi.rx(np.pi/2,Q-1-q)
            pauli_qs.append(1)
            Z_label += 'Z'
        if p == 'Z':
            pauli_qs.append(1)
            Z_label += 'Z'
        if p == 'I':
            pauli_qs.append(0)
            Z_label += 'I'
    psi.measure(psi.qubits,psi.clbits)
    return psi,pauli_qs
    
def collect_circuits(H_paulis,cir_in):
    circs = []
    labels = []
    paulis = list(H_paulis.keys())
    for p in paulis:
        circ,label = apply_rotations(p,cir_in)
        circs.append(circ)
        labels.append([H_paulis[p],label])
    return circs,labels

Overwriting Define_Pauli_Measure.py


In [84]:
import scipy.linalg as ln
from Define_Paulis import X,Y,Z,Mdot
import numpy as np
from Define_Ansatz import cir_A

cir = cir_A(0.5,[[0.1,0.2],[0.3,0.4]])

H = Z(0,2) + Z(1,2) + 0.3*Mdot([X(0,2),X(1,2)])

e,y = ln.eig(H)
arg = np.argsort(e)
psi = np.conjugate(np.transpose(y))

def H_paulis(m,k):
    h = {'ZI':m,'IZ':m,'XX':k}
    return h

In [85]:
measure_E(H_paulis(1,0.3), cir, method = 'matrix')

(1.4488787666369707+0j)

In [86]:
rho = translate(cir)
np.trace( Mdot([H,rho]) )

(1.4488787666369707+0j)

In [87]:
from Define_Ansatz import cir_A

cir_in = cir_A(0.5,[[0.1,0.2],[0.3,0.4]])

circs, labels = collect_circuits(H_paulis(1,0.3),cir_in)


print(labels[2])
circs[2].draw()

[0.3, [1, 1]]


┌─────────────┐┌───┐ ░ ┌─┐    ░ ┌─────────┐     ┌─────────┐     »
q26_0: ┤ Ry(0.92657) ├┤ Z ├─░─┤M├────░─┤ Ry(0.1) ├──■──┤ Ry(0.3) ├──■──»
       └─────────────┘└───┘ ░ └╥┘┌─┐ ░ ├─────────┤┌─┴─┐├─────────┤┌─┴─┐»
q26_1: ─────────────────────░──╫─┤M├─░─┤ Ry(0.2) ├┤ X ├┤ Ry(0.4) ├┤ X ├»
                            ░  ║ └╥┘ ░ └─────────┘└───┘└─────────┘└───┘»
c26: 2/════════════════════════╩══╩════════════════════════════════════»
                               0  1                                    »
«       ┌──────────┐┌─┐   
«q26_0: ┤ Ry(-π/2) ├┤M├───
«       ├──────────┤└╥┘┌─┐
«q26_1: ┤ Ry(-π/2) ├─╫─┤M├
«       └──────────┘ ║ └╥┘
«c26: 2/═════════════╩══╩═
«                    0  1

In [88]:
pauli_qs = labels[0]
rho = translate(circs[0])
r = measure_rho(rho)
z_measure = 0
for key in list(r.keys()):
    n = 0
    for q in range(len(key)):
        if key[q] == '1' and pauli_qs[q] == 1:
            n += 1
    z_measure += (-1)**n * r[key] 
    
z_measure

(0.8251161559642037+0j)

In [89]:
measure_pauli(paulis[0],cir)

(0.8251161559642037+0j)

In [90]:
measure_E(H_paulis(1,0.3), cir)

(1.4488787666369707+0j)

In [92]:
paulis = list(H_paulis(1,0.3).keys()) 
E = 0
for p in range( len(paulis) ):
    pauli_qs = labels[p][1]
    rho = translate(circs[p])
    r = measure_rho(rho)
    z_measure = 0
    for key in list(r.keys()):
        n = 0
        for q in range(len(key)):
            if key[q] == '1' and pauli_qs[q] == 1:
                n += 1
        z_measure += (-1)**n * r[key] 
    E += labels[p][0]*z_measure
    #print(paulis[p],H_paulis(1,0.3)[paulis[p]],z_measure)
        

E

(1.4488787666369707+0j)